In [2]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install einops
!pip install faiss-gpu
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub

!pip install --upgrade --quiet  wikipedia
!pip install --upgrade --quiet  arxiv
!pip install --upgrade --quiet  pymupdf

!pip install xmltodict

!pip install cohere
!pip install -U langchain-cohere

In [3]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets
!pip -q install langchain
!pip install einops
!pip install faiss-gpu
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm

In [4]:
import os
import torch
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig

In [5]:
# Load the XSum dataset
dataset = load_dataset("xsum")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [6]:
%%capture

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         llm_int4_enable_fp32_cpu_offload=True)


llm_mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Unused kwargs: ['llm_int4_enable_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [7]:
mistral_pipe = pipeline(
    "text-generation",
    model=llm_mistral_model,
    tokenizer=llm_mistral_tokenizer,
    max_length=1000,
    temperature=0.6,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2
)
mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id


In [8]:
mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [9]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """[INST] Give me a short one-sentence summarization with the following context: {context}[/INST]"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | mistral_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(summary_id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    for example in tqdm(dataset):
        summary_w_prompt = self.rag_chain.invoke({"query": example['summary'], "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = summary_w_prompt.split("[/INST] ",1)[1]
        results.append(summary)
    return results

In [10]:
rag_model = RAG_Summarization_Model()
rag_model.fit(dataset['validation'])

In [ ]:
%%capture
predictions = rag_model.predict(dataset['validation'], k=3, fetch_k=20)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_to

In [ ]:
references = [example['summary'] for example in dataset['validation']]

In [ ]:
!pip install nltk

In [ ]:
# prompt: calculate bleu score given predictions and references where each prediction has a corresponding reference

from nltk.translate.bleu_score import sentence_bleu

# Calculate BLEU scores
bleu_scores = []
for ref, pred in zip(references, predictions):
    reference = [ref.split()]  # Reference needs to be a list of lists
    candidate = pred.split()
    score = sentence_bleu(reference, candidate)
    bleu_scores.append(score)

# Print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", average_bleu)
